In [1]:
import os
import sys
import pickle
import json
import pandas as pd


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

from src.common.entities import *

os.getcwd()


path_to_ph = './test_data/policy_holders.csv'
path_to_policies = './test_data/policyinfo.csv'

/home/lytvyn/projects/jetty/jetty-migration


#### Pandas from SnowFlake

https://stackoverflow.com/questions/53114687/how-can-i-populate-a-pandas-dataframe-with-the-result-of-a-snowflake-sql-query

# Snowflake Connector

In [2]:
import snowflake.connector
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
with open("../../../../.ssh/sf_synpulse_lytvyn.p8", "rb") as key:
    p_key= serialization.load_pem_private_key(
        key.read(),
        password=b"12345678910",
        backend=default_backend()
    )


pkb = p_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())

ctx = snowflake.connector.connect(
    user='OLEKSANDER.LYTVYN@JETTY.COM',
    account='fia64738.us-east-1',
    private_key=pkb,
    warehouse="COMPUTE_WH",
    database="STAGE",
    schema="PAS_MIGRATION"
)

cs = ctx.cursor()




In [4]:
res = cs.execute("SELECT * FROM RAW.TEST_POLICY_MIGRATION.POLICIES LIMIT 1000")

df = cs.fetch_pandas_all()

# for c in res:
    # print(c)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   POLICY_NUMBER               1000 non-null   object 
 1   POLICY_TERM                 1000 non-null   int8   
 2   MEMBER_ID                   1000 non-null   object 
 3   SCORE                       99 non-null     object 
 4   TYPE                        1000 non-null   object 
 5   BILLING_SCHEDULE            1000 non-null   object 
 6   EFFECTIVE_DATE              1000 non-null   object 
 7   INCEPTION_DATE              1000 non-null   object 
 8   POLICY_STATUS               1000 non-null   object 
 9   CANCELLATION_REASON         185 non-null    object 
 10  AUTO_PAY                    1000 non-null   bool   
 11  IS_REWRITTEN                1000 non-null   bool   
 12  AUTO_RENEWAL_STATUS         1000 non-null   object 
 13  IS_RENEWAL                  1000 n

# Serializaing PolicyHolders

In [5]:
res = cs.execute("SELECT * FROM RAW.TEST_POLICY_MIGRATION.POLICY_HOLDERS LIMIT 1000")

ph_df = cs.fetch_pandas_all()
ph_df.head()

MEMBER_ID        NAME       EMAIL   DOB  \
0  3ae0affe-8cb5-422c-8e6f-6a4ded100547  **********  **********  None   
1  e133f060-ada4-4604-a810-2c12268ef527  **********  **********  None   
2  d74f4f6b-5520-4cda-a7bb-0f01b5a72869  **********  **********  None   
3  48f1d4cb-53eb-4f39-b94c-eb8fbf0ccd0a  **********  **********  None   
4  e053c3e1-9eb9-4cb4-89d4-020d39dee943  **********  **********  None   

  PHONE_NUMBER CREDIT_SCORE STRIPE_PUBLISHABLE_KEY_AND_PAYMENT_METHODS  \
0   **********   **********                                       None   
1   **********   **********                                       None   
2   **********   **********                                       None   
3   **********   **********                                       None   
4   **********   **********                                       None   

  STRIPE_CUSTOMER_PROFILE_IDS  
0                        None  
1                        None  
2                        None  
3                        None  
4                        None

'**********'

In [20]:
from datetime import date, datetime


def resolve_date(date_str):
    try:
        date = datetime.strptime(date_str, "%Y-%m-%d").date()
    except Exception as e:
        date = datetime.fromtimestamp(965127141).date()
    finally:
        return date

def resolve_stripe(stripe_str):
    if pd.notna(stripe_str):
        stripe_list = stripe_str.split("|")
        return stripe_list[0], stripe_list[1]
    else:
        return None, None
    
resolve_scid = lambda x: x.split(",")[0] if pd.notna(x) else None
resolve_email = lambda x: x if pd.notna(x) else ""
    

def create_policy_holder(row):
    first_name, last_name = row["NAME"].split(' ')[0], row["NAME"].split(' ')[1]
    strip_publishable_key, stripe_source_id = resolve_stripe(row["STRIPE_PUBLISHABLE_KEY_AND_PAYMENT_METHODS"])
    return PolicyHolder(
        first_name=first_name,
        last_name=last_name,
        email=resolve_email(row["EMAIL"]),
        date_of_birth=resolve_date(row["DOB"]),
        custom_member_id=row["MEMBER_ID"],
        strip_publishable_key=strip_publishable_key,
        stripe_source_id=stripe_source_id,
        stripe_customer_id=resolve_scid(row["STRIPE_CUSTOMER_PROFILE_IDS"])
    )

In [22]:
phs = ph_df.apply(create_policy_holder, axis=1)
phs.iloc[0].model_dump()

{'first_name': 'FirstName',
 'last_name': 'LastName',
 'email': 'email@gmail.com',
 'phone_number': None,
 'date_of_birth': datetime.date(2000, 8, 1),
 'custom_member_id': 'mem_af06b691bda3403480ff372d9a2f0028',
 'ofac_outcome': None,
 'ofac_reason': None,
 'ofac_date': None,
 'stripe_publishable_key': None,
 'stripe_customer_id': None,
 'stripe_source_id': None,
 'stripe_customer_id_2': None,
 'stripe_customer_id_3': None,
 'stripe_customer_id_4': None,
 'stripe_customer_id_5': None}

# Serializaing Policies

## Jetty Deposit Dynamic/Fixed

In [82]:
p_df = pd.read_csv("./test_data/policyinfo_new.csv", dtype_backend='pyarrow')


/tmp/ipykernel_31813/3110123836.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  p_df = pd.read_csv("./test_data/policyinfo_new.csv", dtype_backend='pyarrow')


In [87]:
new_pd = p_df[pd.notna(p_df["ADDITIONAL_INTERESTS"])]

# json.loads(new_pd.iloc[1]["ADDITIONAL_INTERESTS"])
new_pd.iloc[0]["ADDITIONAL_INTERESTS"]

'[\n  {\n    "address_city": "Franklin ",\n    "address_line_1": "3100 aspen grove drive",\n    "address_line_2": "",\n    "address_state": "TN",\n    "address_zip": "37067",\n    "email": "Cianna.Williams@berkshirecommunities.com",\n    "name": "Wyndchase aspen grove Berkshire properties"\n  }\n]'

In [80]:
def check(my_string):
    brackets = ['()', '{}', '[]']
    while any(x in my_string for x in brackets):
        for br in brackets:
            my_string = my_string.replace(br, '')
    return not my_string

string = new_pd.iloc[0]["NON_PRIMARY_POLICY_HOLDERS"]
print(string, "-", "Balanced"
      if check(string) else "Unbalanced")

[  
  { 
  "email": "email@email.com",  
  "name": "fname lname", 
  "phone": "(111) 111-1111"  
  },
  "email": "email@email.com",  
  "name": "fname lname", 
  "phone": "(111) 111-1111"  
  }
 ] - Unbalanced


In [65]:

json.loads(new_df.iloc[2]["ADDITIONAL_INTERESTS"].replace("“", '"').replace("”", '"'))
# new_df

[{'address_city': 'Panama City Beach',
  'address_line_1': '10400 Panama City Beach Parkway',
  'address_state': 'FL',
  'address_zip': '32407',
  'email': 'email@email.com',
  'name': 'name, LLC'},
 {'address_city': 'Panama City Beach',
  'address_line_1': '10400 Panama City Beach Parkway',
  'address_state': 'FL',
  'address_zip': '32407',
  'email': 'email@email.com',
  'name': 'name, LLC'}]

In [28]:
import json
from src.common.entities.policy import *

def create_building_details(row):
    bd_original = json.loads(row["BUILDING_DETAILS"])

    building_details = DepositPolicyBuildingFieldGroupItem(
        fieldName = "building_details",
        fieldValues= BuidlingDetails(
            address_line_1=bd_original["address_line_1"],
            address_city=bd_original["address_city"],
            address_state=bd_original["address_state"],
            address_zip=bd_original["address_zip"],
            building_id=bd_original["building_id"],
            sfdc_property_id_at_purchase=bd_original["sfdc_property_id_at_purchase"],
        )
    )
    return building_details

def create_addl_interests(row):
    addl_interest_list = json.loads(row["ADDITIONAL_INTERESTS"].replace("“", '"').replace("”", '"'))
    output = []
    for addl_interest in addl_interest_list:
        output.append(
            DepositPolicyBuildingFieldGroupItem(
        fieldName="addl_interests",
        fieldValues=
            AddlInterest(
                addl_interest_name=addl_interest["name"],
                addl_interest_email=addl_interest["email"],
                addl_interest_address_city=addl_interest["address_city"],
                addl_interest_address_line_1=addl_interest["address_line_1"],
                addl_interest_address_state=addl_interest["address_state"],
                addl_interest_address_zip=addl_interest["address_zip"],
            )
        )
    )            
    return output



def create_deposit_policy(row, ph_locator=""):
    fv = DepositPolicyFieldValues(
        custom_policy_number=row["POLICY_NUMBER"],
        is_renewal=row["IS_RENEWAL"],
        policy_term=row["POLICY_TERM"],
        rater_source="BrightCore",
        rater_id=row["RATE_VERSION"],
        # policy_cancellation_reason=row["CANCELLATION_REASON"],
        credit_score=row["CREDIT_SCORE"],
        auto_renewal_status=row["AUTO_RENEWAL_STATUS"].lower(),
        #remove
        number_of_units_in_building='three units'
    )

    building_details = create_building_details(row) 
    addl_interests = create_addl_interests(row)
    

    return DepositPolicy(
        policyHolderLocator=ph_locator,
        productName=row["TYPE"].replace(" ",""),
        policyStartTimestamp=row["EFFECTIVE_DATE"],
        fieldValues=fv,
        fieldGroups=[
            building_details,
            *addl_interests,
            
            
        ]
    )

create_deposit_policy(p_df.iloc[0])

ValidationError: 4 validation errors for DepositPolicy
policyholderLocator
  Field required [type=missing, input_value={'policyHolderLocator': '...holder_locator_id=None)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.1/v/missing
policyStartTimestamp
  Value error, invalid literal for int() with base 10: '2021-06-07' [type=value_error, input_value='2021-06-07', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1/v/value_error
paymentScheduleName
  Field required [type=missing, input_value={'policyHolderLocator': '...holder_locator_id=None)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.1/v/missing
fieldGroups
  Field required [type=missing, input_value={'policyHolderLocator': '...holder_locator_id=None)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.1/v/missing